# PART B
## Mayank Raj
### Using German dataset and German-Bert Model

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Data manipulation libraries
import pandas as pd
import numpy as np

# PyTorch utilities
import torch
from torch.utils.data import DataLoader

# Transformers and optimization
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

# Preprocessing and model evaluation libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [ ]:
# Load and split the dataset
data = pd.read_csv('/kaggle/input/german-rap-de/final_raps_de.csv')

# Define split ratio and create training/testing subsets
train_df, test_df = np.split(data, [int(0.8 * len(data))])

# Encode labels in 'grade' column
train_df['grade'] = label_encoder.fit_transform(train_df['grade'])
test_df['grade'] = label_encoder.transform(test_df['grade'])


/tmp/ipykernel_30/1848729433.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['grade'] = label_encoder.fit_transform(train_df['grade'])
/tmp/ipykernel_30/1848729433.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['grade'] = label_encoder.transform(test_df['grade'])


In [5]:
model_name = 'bert-base-german-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
train_encodings = tokenizer(train_df['lyrics'].tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_df['lyrics'].tolist(), truncation=True, padding=True, max_length=128)

In [ ]:
# Define a dataset class for model input
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)  # Convert labels to tensor directly

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Retrieve each item and add 'labels' key directly
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items(), 'labels': self.labels[idx]}

# Instantiate datasets for training and testing
train_dataset = TextDataset(train_encodings, train_df['grade'].values)
test_dataset = TextDataset(test_encodings, test_df['grade'].values)


In [8]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

In [9]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))
model.to('cuda' if torch.cuda.is_available() else 'cpu')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# Initialize optimizer with specific learning rate
optimizer = torch.optim.AdamW(params=model.parameters(), lr=5e-5)


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Train the model
for epoch in range(3):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        # Move batch to device in a single line
        inputs = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1} completed.')


Epoch 1 completed.
Epoch 2 completed.
Epoch 3 completed.


In [ ]:
# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Evaluate on the test set
model.eval()
val_preds, val_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        inputs = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**inputs)
        preds = torch.argmax(outputs.logits, dim=1)
        val_preds.append(preds.cpu().numpy())
        val_labels.append(batch['labels'].cpu().numpy())

val_accuracy = np.mean(np.array(val_preds) == np.array(val_labels))
print(f'Validation Accuracy: {val_accuracy:.4f}')


Validation Accuracy: 0.4787
